In [7]:
import pandas as pd
from pathlib import Path

root = Path("..")

df = pd.read_parquet(root / "data/merged/train_audio_lyrics_final.parquet")

# শুধু audio-related column রাখি
df_audio_only = df[["spotify_id", "mp3_path"]].copy()

out = root / "data/merged/train_audio_only.parquet"
df_audio_only.to_parquet(out, index=False)

print("✅ saved:", out)
print("rows:", len(df_audio_only))


✅ saved: ..\data\merged\train_audio_only.parquet
rows: 1432


In [10]:
import numpy as np
import librosa
from tqdm import tqdm

def extract_mfcc(mp3_path, sr=22050, n_mfcc=40, max_len=300):
    y, sr = librosa.load(mp3_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc).T  # (time, n_mfcc)
    if mfcc.shape[0] < max_len:
        mfcc = np.vstack([mfcc, np.zeros((max_len - mfcc.shape[0], n_mfcc))])
    else:
        mfcc = mfcc[:max_len, :]
    return mfcc.flatten()


In [11]:
from pathlib import Path
root = Path("..")

X_audio = []
kept = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="MFCC"):
    try:
        X_audio.append(extract_mfcc(row["mp3_path"]))
        kept.append(i)
    except:
        pass

X_audio = np.array(X_audio)

# save features
np.save(root / "data/merged/X_audio_mfcc.npy", X_audio)

# save kept list (to align later)
df_kept = df.loc[kept].reset_index(drop=True)
df_kept.to_parquet(root / "data/merged/train_audio_only_final.parquet", index=False)

print("✅ X_audio shape:", X_audio.shape)
print("✅ saved: data/merged/X_audio_mfcc.npy")
print("✅ saved: data/merged/train_audio_only_final.parquet")


MFCC: 100%|██████████| 1432/1432 [02:01<00:00, 11.79it/s]

✅ X_audio shape: (1432, 12000)
✅ saved: data/merged/X_audio_mfcc.npy
✅ saved: data/merged/train_audio_only_final.parquet


In [1]:
import pandas as pd
from pathlib import Path

root = Path("..")
df = pd.read_parquet(root / "data/merged/train_audio_lyrics_final.parquet")

# mp3 path বানানো
df["mp3_path"] = df["spotify_id"].astype(str).apply(lambda x: str(root / "data/audio_mp3" / f"{x}.mp3"))

print("rows:", len(df))
df[["spotify_id","mp3_path"]].head(3)


rows: 1432


,spotify_id,mp3_path
0,0z4sFQ2yzLbjzSsNC5qAkD,..\data\audio_mp3\0z4sFQ2yzLbjzSsNC5qAkD.mp3
1,0qQBuGsT79rKtmmfC0V38x,..\data\audio_mp3\0qQBuGsT79rKtmmfC0V38x.mp3
2,169mBDYNy6SFx2gb28pG1e,..\data\audio_mp3\169mBDYNy6SFx2gb28pG1e.mp3


In [2]:
import numpy as np
import librosa
from tqdm import tqdm

def extract_mfcc(mp3_path, sr=22050, n_mfcc=40, max_len=300):
    y, sr = librosa.load(mp3_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc).T
    if mfcc.shape[0] < max_len:
        mfcc = np.vstack([mfcc, np.zeros((max_len - mfcc.shape[0], n_mfcc))])
    else:
        mfcc = mfcc[:max_len, :]
    return mfcc.flatten()

X_audio = []
kept_rows = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="MFCC"):
    try:
        X_audio.append(extract_mfcc(row["mp3_path"]))
        kept_rows.append(i)
    except:
        pass

X_audio = np.array(X_audio)
df_kept = df.loc[kept_rows].reset_index(drop=True)

np.save(root / "data/merged/X_audio_mfcc.npy", X_audio)
df_kept.to_parquet(root / "data/merged/train_audio_lyrics_final_kept.parquet", index=False)

print("X_audio shape:", X_audio.shape)
print("kept df rows:", len(df_kept))
print("saved: X_audio_mfcc.npy + train_audio_lyrics_final_kept.parquet")


MFCC: 100%|██████████| 1432/1432 [02:15<00:00, 10.55it/s]


X_audio shape: (1432, 12000)
kept df rows: 1432
saved: X_audio_mfcc.npy + train_audio_lyrics_final_kept.parquet
